In [1]:
using BenchmarkTools

In [2]:
## Basic RBC model with full depreciation
#
# Jesus Fernandez-Villaverde
# Haverford, July 29, 2013

function main()

    ##  1. Calibration

    α = 1/3     # Elasticity of output w.r.t. capital
    β = 0.95    # Discount factor

    # Productivity values
    vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

    # Transition matrix
    mTransition   = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (α*β)^(1/(1-α))
    outputSteadyState = capitalSteadyState^α
    consumptionSteadyState = outputSteadyState-capitalSteadyState

    println("Output = ",outputSteadyState," Capital = ",capitalSteadyState," Consumption = ",consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital = collect(0.5*capitalSteadyState:0.00001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

    mOutput           = zeros(nGridCapital,nGridProductivity)
    mValueFunction    = zeros(nGridCapital,nGridProductivity)
    mValueFunctionNew = zeros(nGridCapital,nGridProductivity)
    mPolicyFunction   = zeros(nGridCapital,nGridProductivity)
    expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

    mOutput = (vGridCapital.^α)*vProductivity;

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    while(maxDifference > tolerance)
        expectedValueFunction = mValueFunction*mTransition';

        for nProductivity in 1:nGridProductivity

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 1

            for nCapital in 1:nGridCapital

                valueHighSoFar = -1000.0
                capitalChoice  = vGridCapital[1]

                for nCapitalNextPeriod in gridCapitalNextPeriod:nGridCapital

                    consumption = mOutput[nCapital,nProductivity]-vGridCapital[nCapitalNextPeriod]
                    valueProvisional = (1-β)*log(consumption)+β*expectedValueFunction[nCapitalNextPeriod,nProductivity]

                    if (valueProvisional>valueHighSoFar)
                	       valueHighSoFar = valueProvisional
                	       capitalChoice = vGridCapital[nCapitalNextPeriod]
                	       gridCapitalNextPeriod = nCapitalNextPeriod
                    else
                	       break # We break when we have achieved the max
                    end

                end

                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity] = capitalChoice

            end

        end
        
        maxDifference     = maximum(abs.(mValueFunctionNew-mValueFunction))
        mValueFunction, mValueFunctionNew = mValueFunctionNew, mValueFunction
        
        iteration = iteration+1
        if mod(iteration,10)==0 || iteration == 1
            println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
        end

    end

    println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
    println(" ")
    println(" My check = ", mPolicyFunction[1000,3])
    println(" My check = ", mValueFunction[1000,3])

end

main (generic function with 1 method)

In [3]:
main()

Output = 0.5627314338711378 Capital = 0.178198287392527 Consumption = 0.3845331464786108
 Iteration = 1 Sup Diff = 0.05274159340733661
 Iteration = 10 Sup Diff = 0.031346949265852075
 Iteration = 20 Sup Diff = 0.01870345989335709
 Iteration = 30 Sup Diff = 0.01116551203397076
 Iteration = 40 Sup Diff = 0.00666854170813258
 Iteration = 50 Sup Diff = 0.003984292748717033
 Iteration = 60 Sup Diff = 0.0023813118039327508
 Iteration = 70 Sup Diff = 0.0014236586450983024
 Iteration = 80 Sup Diff = 0.0008513397747205165
 Iteration = 90 Sup Diff = 0.0005092051752288995
 Iteration = 100 Sup Diff = 0.00030462324421465237
 Iteration = 110 Sup Diff = 0.00018226485632300005
 Iteration = 120 Sup Diff = 0.00010906950872624499
 Iteration = 130 Sup Diff = 6.527643736320421e-5
 Iteration = 140 Sup Diff = 3.907108211997912e-5
 Iteration = 150 Sup Diff = 2.3388077119990136e-5
 Iteration = 160 Sup Diff = 1.4008644637186762e-5
 Iteration = 170 Sup Diff = 8.391317202871562e-6
 Iteration = 180 Sup Diff = 5.02

In [4]:
@time main()

Output = 0.5627314338711378 Capital = 0.178198287392527 Consumption = 0.3845331464786108
 Iteration = 1 Sup Diff = 0.05274159340733661
 Iteration = 10 Sup Diff = 0.031346949265852075
 Iteration = 20 Sup Diff = 0.01870345989335709
 Iteration = 30 Sup Diff = 0.01116551203397076
 Iteration = 40 Sup Diff = 0.00666854170813258
 Iteration = 50 Sup Diff = 0.003984292748717033
 Iteration = 60 Sup Diff = 0.0023813118039327508
 Iteration = 70 Sup Diff = 0.0014236586450983024
 Iteration = 80 Sup Diff = 0.0008513397747205165
 Iteration = 90 Sup Diff = 0.0005092051752288995
 Iteration = 100 Sup Diff = 0.00030462324421465237
 Iteration = 110 Sup Diff = 0.00018226485632300005
 Iteration = 120 Sup Diff = 0.00010906950872624499
 Iteration = 130 Sup Diff = 6.527643736320421e-5
 Iteration = 140 Sup Diff = 3.907108211997912e-5
 Iteration = 150 Sup Diff = 2.3388077119990136e-5
 Iteration = 160 Sup Diff = 1.4008644637186762e-5
 Iteration = 170 Sup Diff = 8.391317202871562e-6
 Iteration = 180 Sup Diff = 5.02

In [5]:
# same as above except without println statements
function main2()

    ##  1. Calibration

    α = 1/3     # Elasticity of output w.r.t. capital
    β = 0.95    # Discount factor

    # Productivity values
    vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

    # Transition matrix
    mTransition   = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (α*β)^(1/(1-α))
    outputSteadyState = capitalSteadyState^α
    consumptionSteadyState = outputSteadyState-capitalSteadyState

#     println("Output = ",outputSteadyState," Capital = ",capitalSteadyState," Consumption = ",consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital = collect(0.5*capitalSteadyState:0.00001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

    mOutput           = zeros(nGridCapital,nGridProductivity)
    mValueFunction    = zeros(nGridCapital,nGridProductivity)
    mValueFunctionNew = zeros(nGridCapital,nGridProductivity)
    mPolicyFunction   = zeros(nGridCapital,nGridProductivity)
    expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

    mOutput = (vGridCapital.^α)*vProductivity;

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    while(maxDifference > tolerance)
        expectedValueFunction = mValueFunction*mTransition';

        for nProductivity in 1:nGridProductivity

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 1

            for nCapital in 1:nGridCapital

                valueHighSoFar = -1000.0
                capitalChoice  = vGridCapital[1]

                for nCapitalNextPeriod in gridCapitalNextPeriod:nGridCapital

                    consumption = mOutput[nCapital,nProductivity]-vGridCapital[nCapitalNextPeriod]
                    valueProvisional = (1-β)*log(consumption)+β*expectedValueFunction[nCapitalNextPeriod,nProductivity]

                    if (valueProvisional>valueHighSoFar)
                	       valueHighSoFar = valueProvisional
                	       capitalChoice = vGridCapital[nCapitalNextPeriod]
                	       gridCapitalNextPeriod = nCapitalNextPeriod
                    else
                	       break # We break when we have achieved the max
                    end

                end

                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity] = capitalChoice

            end

        end
        
        maxDifference     = maximum(abs.(mValueFunctionNew-mValueFunction))
        mValueFunction, mValueFunctionNew = mValueFunctionNew, mValueFunction
        
        iteration = iteration+1
#         if mod(iteration,10)==0 || iteration == 1
#             println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#         end

    end

#     println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#     println(" ")
#     println(" My check = ", mPolicyFunction[1000,3])
#     println(" My check = ", mValueFunction[1000,3])

end

main2 (generic function with 1 method)

In [6]:
main2()

In [7]:
@benchmark main2() seconds=10

BenchmarkTools.Trial: 
  memory estimate:  528.54 MiB
  allocs estimate:  1563
  --------------
  minimum time:     1.942 s (3.28% GC)
  median time:      1.987 s (3.34% GC)
  mean time:        1.989 s (3.90% GC)
  maximum time:     2.055 s (3.17% GC)
  --------------
  samples:          6
  evals/sample:     1

In [8]:
# use Libm log function
function main3()

    ##  1. Calibration

    α = 1/3     # Elasticity of output w.r.t. capital
    β = 0.95    # Discount factor

    # Productivity values
    vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

    # Transition matrix
    mTransition   = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (α*β)^(1/(1-α))
    outputSteadyState = capitalSteadyState^α
    consumptionSteadyState = outputSteadyState-capitalSteadyState

#     println("Output = ",outputSteadyState," Capital = ",capitalSteadyState," Consumption = ",consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital = collect(0.5*capitalSteadyState:0.00001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

    mOutput           = zeros(nGridCapital,nGridProductivity)
    mValueFunction    = zeros(nGridCapital,nGridProductivity)
    mValueFunctionNew = zeros(nGridCapital,nGridProductivity)
    mPolicyFunction   = zeros(nGridCapital,nGridProductivity)
    expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

    mOutput = (vGridCapital.^α)*vProductivity;

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    while(maxDifference > tolerance)
        expectedValueFunction = mValueFunction*mTransition';

        for nProductivity in 1:nGridProductivity

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 1

            for nCapital in 1:nGridCapital

                valueHighSoFar = -1000.0
                capitalChoice  = vGridCapital[1]

                for nCapitalNextPeriod in gridCapitalNextPeriod:nGridCapital

                    consumption = mOutput[nCapital,nProductivity]-vGridCapital[nCapitalNextPeriod]
                    valueProvisional = (1-β)*Base.Math.JuliaLibm.log(consumption)+β*expectedValueFunction[nCapitalNextPeriod,nProductivity]

                    if (valueProvisional>valueHighSoFar)
                	       valueHighSoFar = valueProvisional
                	       capitalChoice = vGridCapital[nCapitalNextPeriod]
                	       gridCapitalNextPeriod = nCapitalNextPeriod
                    else
                	       break # We break when we have achieved the max
                    end

                end

                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity] = capitalChoice

            end

        end
        
        maxDifference     = maximum(abs.(mValueFunctionNew-mValueFunction))
        mValueFunction, mValueFunctionNew = mValueFunctionNew, mValueFunction
        
        iteration = iteration+1
#         if mod(iteration,10)==0 || iteration == 1
#             println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#         end

    end

#     println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#     println(" ")
#     println(" My check = ", mPolicyFunction[1000,3])
#     println(" My check = ", mValueFunction[1000,3])

end

main3 (generic function with 1 method)

In [9]:
main3()

In [10]:
@benchmark main3() seconds=10

BenchmarkTools.Trial: 
  memory estimate:  528.54 MiB
  allocs estimate:  1563
  --------------
  minimum time:     1.534 s (4.03% GC)
  median time:      1.869 s (4.43% GC)
  mean time:        1.872 s (5.13% GC)
  maximum time:     2.138 s (4.05% GC)
  --------------
  samples:          6
  evals/sample:     1

In [11]:
@benchmark main3() seconds=10

BenchmarkTools.Trial: 
  memory estimate:  528.54 MiB
  allocs estimate:  1563
  --------------
  minimum time:     1.534 s (4.23% GC)
  median time:      1.847 s (4.42% GC)
  mean time:        1.835 s (5.36% GC)
  maximum time:     2.103 s (9.04% GC)
  --------------
  samples:          6
  evals/sample:     1

In [12]:
@profile main3()

In [13]:
Profile.print()

1204 ./task.jl:335; (::IJulia.##14#17)()
 1204 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  1204 .../Compat/src/Compat.jl:488; (::Compat.#inner#17{Array{Any,1}...
   1203 ...c/execute_request.jl:158; execute_request(::ZMQ.Socket, :...
    1203 ...Compat/src/Compat.jl:174; include_string(::Module, ::Str...
     1203 ./loading.jl:522; include_string(::String, ::String)
      1203 ./<missing>:?; anonymous
       1203 ./profile.jl:23; macro expansion
        1   ./In[8]:37; main3()
         1 ./array.jl:227; fill!(::Array{Float64,2}, ::F...
        28  ./In[8]:52; main3()
         28 ./linalg/matmul.jl:189; A_mul_Bt(::Array{Float64,2}, ...
          15 ./linalg/matmul.jl:191; A_mul_Bt!
           15 ./linalg/matmul.jl:367; gemm_wrapper!(::Array{Float...
            13 ./linalg/blas.jl:1027; gemm!(::Char, ::Char, ::Fl...
        2   ./In[8]:59; main3()
        27  ./In[8]:62; main3()
        55  ./In[8]:64; main3()
        102 ./In[8]:66; main3()
        655 ./In[8]:67; main3()
  

In [14]:
# inbounds
function main4()

    ##  1. Calibration

    α = 1/3     # Elasticity of output w.r.t. capital
    β = 0.95    # Discount factor

    # Productivity values
    vProductivity = [0.9792 0.9896 1.0000 1.0106 1.0212]

    # Transition matrix
    mTransition   = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    capitalSteadyState = (α*β)^(1/(1-α))
    outputSteadyState = capitalSteadyState^α
    consumptionSteadyState = outputSteadyState-capitalSteadyState

#     println("Output = ",outputSteadyState," Capital = ",capitalSteadyState," Consumption = ",consumptionSteadyState)

    # We generate the grid of capital
    vGridCapital = collect(0.5*capitalSteadyState:0.00001:1.5*capitalSteadyState)

    nGridCapital = length(vGridCapital)
    nGridProductivity = length(vProductivity)

    # 3. Required matrices and vectors

    mOutput           = zeros(nGridCapital,nGridProductivity)
    mValueFunction    = zeros(nGridCapital,nGridProductivity)
    mValueFunctionNew = zeros(nGridCapital,nGridProductivity)
    mPolicyFunction   = zeros(nGridCapital,nGridProductivity)
    expectedValueFunction = zeros(nGridCapital,nGridProductivity)

    # 4. We pre-build output for each point in the grid

    mOutput = (vGridCapital.^α)*vProductivity;

    # 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.0000001
    iteration = 0

    while(maxDifference > tolerance)
        expectedValueFunction = mValueFunction*mTransition';

        for nProductivity in 1:nGridProductivity

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 1

            for nCapital in 1:nGridCapital

                valueHighSoFar = -1000.0
                capitalChoice  = vGridCapital[1]

                for nCapitalNextPeriod in gridCapitalNextPeriod:nGridCapital

                    @inbounds consumption = mOutput[nCapital,nProductivity]-vGridCapital[nCapitalNextPeriod]
                    valueProvisional = (1-β)*Base.Math.JuliaLibm.log(consumption)+β*expectedValueFunction[nCapitalNextPeriod,nProductivity]

                    if (valueProvisional>valueHighSoFar)
                	       valueHighSoFar = valueProvisional
                	       capitalChoice = vGridCapital[nCapitalNextPeriod]
                	       gridCapitalNextPeriod = nCapitalNextPeriod
                    else
                	       break # We break when we have achieved the max
                    end

                end

                mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
                mPolicyFunction[nCapital,nProductivity] = capitalChoice

            end

        end
        
        maxDifference     = maximum(abs.(mValueFunctionNew-mValueFunction))
        mValueFunction, mValueFunctionNew = mValueFunctionNew, mValueFunction
        
        iteration = iteration+1
#         if mod(iteration,10)==0 || iteration == 1
#             println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#         end

    end

#     println(" Iteration = ", iteration, " Sup Diff = ", maxDifference)
#     println(" ")
#     println(" My check = ", mPolicyFunction[1000,3])
#     println(" My check = ", mValueFunction[1000,3])

end

main4 (generic function with 1 method)

In [15]:
main4()

In [19]:
@benchmark main4() seconds=10

BenchmarkTools.Trial: 
  memory estimate:  528.54 MiB
  allocs estimate:  1563
  --------------
  minimum time:     1.552 s (4.61% GC)
  median time:      1.663 s (4.63% GC)
  mean time:        1.685 s (5.69% GC)
  maximum time:     1.908 s (10.33% GC)
  --------------
  samples:          6
  evals/sample:     1